# Job Application Toolkit

A streamlined toolkit for email sending and cover letter generation.

## Configuration and Libraries

In [22]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from dotenv import load_dotenv

try:
    from docx2pdf import convert
    PDF_CONVERSION_AVAILABLE = True
except ImportError:
    PDF_CONVERSION_AVAILABLE = False

load_dotenv()

GMAIL_EMAIL = os.getenv('GMAIL_EMAIL')
GMAIL_PASSWORD = os.getenv('GMAIL_PASSWORD')
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

FULL_NAME = "Felix Lau Pangestu"
PHONE_NUMBER = os.getenv('PHONE_NUMBER')
LINKEDIN_URL = "linkedin.com/in/jfelixpangestu/"

## Template Functions

In [23]:
def coverLetterBody(company_name, role, industry, company_specific_interest=None, referral_name=None):
    """Generate cover letter body with optimal length"""
    
    if referral_name:
        opening = f"I am writing to apply for the {role} position at {company_name}. {referral_name} spoke highly of the team and culture, which reinforced my strong interest in joining your organization."
    else:
        opening = f"I am writing to express my enthusiasm for the {role} position at {company_name}."
    
    if company_specific_interest:
        interest_statement = company_specific_interest
    else:
        interest_statements = {
            "investment banking": "Your firm's reputation for excellence and commitment to helping clients navigate complex financial challenges",
            "technology": "Your innovative approach to solving complex problems and building products that impact millions of users",
            "fintech": "Your mission to democratize financial services through technology and create more accessible financial solutions",
            "consulting": "Your collaborative approach to solving business challenges and commitment to client success",
            "asset management": "Your focus on delivering superior investment outcomes and fiduciary responsibility to clients",
            "software engineering": "Your commitment to building scalable, reliable systems and fostering a culture of technical excellence",
            "data science": "Your data-driven approach to solving complex problems and extracting meaningful insights from large datasets",
            "cybersecurity": "Your dedication to protecting digital assets and staying ahead of evolving security threats",
            "private equity": "Your strategic approach to value creation and commitment to transforming businesses for long-term success",
            "hedge funds": "Your sophisticated investment strategies and commitment to delivering alpha through rigorous research and analysis",
            "corporate finance": "Your strategic role in driving business growth through capital allocation and financial planning",
            "risk management": "Your proactive approach to identifying and mitigating financial risks while enabling business growth",
            "quantitative finance": "Your innovative use of mathematical models and technology to solve complex financial problems",
            "product management": "Your user-centric approach to building products that solve real problems and create meaningful value",
            "venture capital": "Your vision for identifying and nurturing innovative companies that shape the future",
            "real estate": "Your expertise in creating value through strategic property investments and development",
            "energy": "Your commitment to sustainable energy solutions and driving the transition to a cleaner future",
            "healthcare": "Your mission to improve patient outcomes and advance medical technology for better lives",
            "pharmaceuticals": "Your dedication to developing life-saving treatments and advancing medical research",
            "aerospace": "Your pioneering work in advancing aviation and space exploration technologies",
            "aviation": "Your pioneering work in advancing aviation technologies and commitment to enhancing the future of air travel",
            "automotive": "Your innovation in transforming mobility and developing next-generation transportation solutions",
            "retail": "Your focus on enhancing customer experiences and adapting to evolving consumer needs",
            "e-commerce": "Your commitment to revolutionizing online commerce and creating seamless digital experiences",
            "telecommunications": "Your role in connecting people globally and building the infrastructure for digital communication",
            "media": "Your dedication to creating compelling content and shaping how people consume information and entertainment",
            "gaming": "Your passion for creating immersive experiences that bring joy and connection to millions of players",
            "education technology": "Your mission to democratize access to quality education through innovative technology solutions",
            "artificial intelligence": "Your pioneering work in developing AI systems that augment human capabilities and solve complex challenges",
            "blockchain": "Your vision for building decentralized systems that create transparency and trust in digital transactions",
            "cloud computing": "Your commitment to enabling digital transformation through scalable and reliable cloud infrastructure",
            "logistics": "Your innovative solutions for optimizing supply chains and enabling global commerce",
            "manufacturing": "Your focus on operational excellence and leveraging technology to drive efficiency and quality",
            "insurance": "Your commitment to protecting individuals and businesses while innovating risk assessment and management",
            "accounting": "Your dedication to financial accuracy and providing strategic insights that drive business decisions",
            "legal services": "Your commitment to providing exceptional legal counsel and navigating complex regulatory environments",
            "non-profit": "Your mission-driven approach to creating positive social impact and addressing important societal challenges",
            "architecture": "Your innovative designs that blend functionality with aesthetic appeal to create inspiring spaces",
            "engineering": "Your commitment to solving real-world challenges through innovative engineering solutions and advancing the standards of quality, safety, and sustainability",
            "market making": "Your pioneering role in providing continuous liquidity across global financial markets and commitment to driving transparency and efficiency through cutting-edge proprietary trading technology"
        }
        interest_statement = interest_statements.get(industry, "Your commitment to excellence and innovative approach to solving complex challenges in your industry")
    
    # Enhanced technical experience paragraph
    experience_para = f"As a final-year Business Computing & Data Analytics student at Hong Kong Baptist University with First Class Honors, I have developed expertise in full-stack web development, database management, and machine learning through internships at organizations like Cathay Pacific and Wizpresso. My experience building scalable applications, optimizing system performance, and working with large datasets has prepared me well for the technical challenges at {company_name}. I've also had the opportunity to lead cross-functional projects and communicate complex technical concepts to stakeholders, experiences that align well with the collaborative nature of your {role} position."
    
    # New academic and leadership paragraph
    leadership_para = f"Beyond technical skills, my academic journey has included research work on large language models and artificial intelligence applications in finance, areas increasingly relevant in today's {industry} landscape. As a Representative of the Senate for the Science Faculty and AWS Student Ambassador, I've developed strong leadership and communication abilities while organizing technical events for hundreds of participants. My entrepreneurial experience founding 'Read the Vision,' a computer vision solution admitted to the HKSTP Ideation Programme, demonstrates my drive to use technology for meaningful impact."
    
    # Enhanced culture fit paragraph
    culture_para = f"What excites me most about {company_name} is the opportunity to work alongside talented professionals who share a passion for {industry} and continuous innovation. I thrive in environments that challenge me to grow, encourage learning, and provide opportunities to contribute to impactful projects. My experience collaborating with diverse teams, combined with my international perspective from my exchange semester at Nanyang Technological University, has prepared me well for {company_name}'s dynamic and collaborative culture."
    
    # Professional closing
    closing = f"I would be thrilled to contribute to {company_name}'s continued success and would welcome the opportunity to discuss how my technical background, leadership experience, and genuine passion for {industry} can add value to your team. Thank you for considering my application."
    
    return f"{opening} {interest_statement} particularly resonates with me, and I am excited about the opportunity to contribute to your team's success.\n\n{experience_para}\n\n{leadership_para}\n\n{culture_para}\n\n{closing}"

def emailLetter(company_name, role, industry=None, company_specific_interest=None, referral_name=None):
    """Generate personalized email content similar to coverLetterBody methodology but concise for email format"""
    
    # Industry-specific opening statements (concise versions)
    industry_openings = {
        "investment banking": f"I am excited to apply for the {role} position at {company_name}, drawn by your firm's reputation for excellence in financial advisory and client-focused solutions.",
        "technology": f"I am writing to express my strong interest in the {role} position at {company_name}, particularly inspired by your innovative approach to solving complex technical challenges.",
        "fintech": f"I am enthusiastic about the {role} opportunity at {company_name}, especially your mission to democratize financial services through cutting-edge technology.",
        "consulting": f"I am excited to apply for the {role} position at {company_name}, attracted by your collaborative approach to solving strategic business challenges.",
        "asset management": f"I am writing to express my interest in the {role} position at {company_name}, drawn by your commitment to delivering superior investment outcomes for clients.",
        "software engineering": f"I am excited about the {role} opportunity at {company_name}, particularly your commitment to building scalable systems and fostering technical excellence.",
        "data science": f"I am enthusiastic about the {role} position at {company_name}, especially your data-driven approach to extracting meaningful insights from complex datasets.",
        "cybersecurity": f"I am writing to apply for the {role} position at {company_name}, drawn by your dedication to protecting digital assets and staying ahead of security threats.",
        "aviation": f"I am excited to apply for the {role} position at {company_name}, particularly inspired by your pioneering work in advancing aviation technologies and enhancing the future of air travel.",
        "aerospace": f"I am writing to express my strong interest in the {role} position at {company_name}, drawn by your pioneering work in advancing aerospace and space exploration technologies.",
        "automotive": f"I am enthusiastic about the {role} opportunity at {company_name}, especially your innovation in transforming mobility and developing next-generation transportation solutions.",
        "healthcare": f"I am excited to apply for the {role} position at {company_name}, inspired by your mission to improve patient outcomes and advance medical technology.",
        "energy": f"I am writing to express my interest in the {role} position at {company_name}, particularly your commitment to sustainable energy solutions and the clean energy transition.",
        "artificial intelligence": f"I am enthusiastic about the {role} opportunity at {company_name}, especially your pioneering work in developing AI systems that solve complex challenges.",
        "engineering": f"I am excited to apply for the {role} position at {company_name}, drawn by your commitment to solving real-world challenges through innovative engineering solutions."
    }
    
    # Get industry-specific opening or use generic one
    if industry and industry.lower() in industry_openings:
        opening = industry_openings[industry.lower()]
    elif company_specific_interest:
        opening = f"I am writing to express my strong interest in the {role} position at {company_name}. {company_specific_interest}"
    elif referral_name:
        opening = f"I am excited to apply for the {role} position at {company_name}. {referral_name} spoke highly of your team and culture, which reinforced my interest in joining your organization."
    else:
        opening = f"I am writing to express my strong interest in the {role} position at {company_name}."
    
    # Concise technical background (email-appropriate length)
    background = "With First Class Honors in my studies, I bring hands-on experience from multiple software engineer internships where I developed full-stack applications, optimized system performance, and worked with large datasets. My technical skills in software development and data analysis, combined with strong project leadership experience, make me well-suited for this role."
    
    # Brief value proposition
    value_prop = f"I am confident that my technical qualifications, practical experience, and passion for {industry if industry else 'innovation'} make me an excellent candidate for this position. I would welcome the opportunity to discuss how I can contribute to {company_name}'s continued success."
    
    # Professional closing
    closing = "Thank you for your consideration. Attached you will find my CV and supporting documents."
    
    signature = "Best regards,\nFelix Lau Pangestu"
    
    return f"""Dear {company_name} Team,

{opening}

{background}

{value_prop}

{closing}

{signature}
"""

## Email Sending Function

In [24]:
def send_email(to_email, subject, body, attachment_path=None):
    """Send email using Gmail SMTP"""
    try:
        msg = MIMEMultipart()
        msg['From'] = GMAIL_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        
        msg.attach(MIMEText(body, 'plain'))
        
        if attachment_path and not isinstance(attachment_path, list):
            attachment_path = [attachment_path]
        elif not attachment_path:
            attachment_path = []

        for single_attachment in attachment_path:
            if os.path.exists(single_attachment):
                with open(single_attachment, "rb") as attachment:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(attachment.read())
                
                encoders.encode_base64(part)
                part.add_header(
                    'Content-Disposition',
                    f'attachment; filename= {os.path.basename(single_attachment)}'
                )
                msg.attach(part)
            else:
                print(f"Attachment not found: {single_attachment}")
        
        context = ssl.create_default_context()
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(GMAIL_EMAIL, GMAIL_PASSWORD)
            server.sendmail(GMAIL_EMAIL, to_email, msg.as_string())
        
        print(f"Email sent successfully to {to_email}")
        return True
        
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return False

## PDF Export Function

In [25]:
def export_cover_letter_to_word_and_pdf(company_name, role, industry, word_filename=None, pdf_filename=None,
                                        company_specific_interest=None, referral_name=None):
    """Export cover letter to Word document and PDF"""
    try:
        cover_letters_dir = "cover_letters"
        if not os.path.exists(cover_letters_dir):
            os.makedirs(cover_letters_dir)
        
        if word_filename is None:
            word_filename = "Cover_Letter.docx"
        if pdf_filename is None:
            pdf_filename = f"cover_letter_{company_name}.pdf"
            
        word_path = word_filename
        pdf_path = os.path.join(cover_letters_dir, pdf_filename)
        
        doc = Document()
        
        sections = doc.sections
        for section in sections:
            section.top_margin = Inches(0.5)
            section.bottom_margin = Inches(0.5)
            section.left_margin = Inches(1)
            section.right_margin = Inches(1)
        
        header = doc.add_paragraph()
        header.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        name_run = header.add_run(FULL_NAME)
        name_run.font.name = 'Times New Roman'
        name_run.font.size = Pt(16)
        name_run.bold = True
        
        header.add_run('\n')
        contact_run = header.add_run(f"{PHONE_NUMBER} | {GMAIL_EMAIL} | {LINKEDIN_URL}")
        contact_run.font.name = 'Times New Roman'
        contact_run.font.size = Pt(11)
    
        # Date and company name in same paragraph
        current_date = datetime.now().strftime("%B %d, %Y")
        date_company_para = doc.add_paragraph(current_date)
        date_company_para.add_run(f"\n{company_name}")
        date_company_para.alignment = WD_ALIGN_PARAGRAPH.LEFT

        salutation_para = doc.add_paragraph("To Whom it May Concern,")
        salutation_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        body_content = coverLetterBody(company_name, role, industry, 
                                     company_specific_interest=company_specific_interest,
                                     referral_name=referral_name)
        
        paragraphs = body_content.strip().split('\n\n')
        
        for i, paragraph in enumerate(paragraphs):
            if paragraph.strip():
                if paragraph.strip().startswith("Thank you for considering") or paragraph.strip().startswith("I would be thrilled"):
                    closing_para = doc.add_paragraph(paragraph.strip())
                    closing_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
                else:
                    body_para = doc.add_paragraph(paragraph.strip())
                    body_para.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

        closing_salutation = doc.add_paragraph("Sincerely,")
        closing_salutation.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        # Add signature image if available
        signature_path = "signature.png"
        if os.path.exists(signature_path):
            signature_para = doc.add_paragraph()
            signature_run = signature_para.add_run()
            signature_run.add_picture(signature_path, width=Inches(2))
            signature_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        else:
            doc.add_paragraph()
            
        name_para = doc.add_paragraph(FULL_NAME)
        name_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        doc.save(word_path)
        
        if PDF_CONVERSION_AVAILABLE:
            try:
                convert(word_path, pdf_path)
                return {'word_path': os.path.abspath(word_path), 'pdf_path': os.path.abspath(pdf_path)}
            except Exception as e:
                print(f"PDF conversion failed: {str(e)}")
                return {'word_path': os.path.abspath(word_path), 'pdf_path': None}
        else:
            return {'word_path': os.path.abspath(word_path), 'pdf_path': None}
        
    except Exception as e:
        print(f"Failed to create documents: {str(e)}")
        return {'word_path': None, 'pdf_path': None}

# Input Variables

In [26]:
# Input variables - modify these for each application
company_name = 'OKX'
position = 'Graduate Software Engineer'
industry = 'fintech'
recipient_email = 'consulting'

In [27]:
# Generate cover letter and export to Word and PDF
# Default values for optional parameters
company_specific_interest = None
referral_name = None

export_result = export_cover_letter_to_word_and_pdf(company_name, position, industry,
                                                   company_specific_interest=company_specific_interest,
                                                   referral_name=referral_name)

word_path = export_result['word_path']
pdf_path = export_result['pdf_path']

  0%|          | 0/1 [00:00<?, ?it/s]

## Generate Content

In [28]:
# Generate email content with improved personalization
# Email details and attachment paths are set automatically
email_subject = f'Application for {position} Position'
attachment_paths = ['Felix Lau Pangestu_CV.pdf', f'cover_letters\\cover_letter_{company_name}.pdf', 'Transcript_HKBU.pdf', 'Recommendation_Letters.pdf']

email_content = emailLetter(company_name, position, industry, 
                           company_specific_interest=company_specific_interest,
                           referral_name=referral_name)

# Generate cover letter body content
cover_letter_body = coverLetterBody(company_name, position, industry, 
                                   company_specific_interest=company_specific_interest,
                                   referral_name=referral_name)

## Send Email

In [29]:
# Uncomment the line below to send the email
# send_email(recipient_email, email_subject, email_content, attachment_paths)

print("Email content generated")
print("Cover letter documents created")
print(f"Ready to send to: {recipient_email}")

Email content generated
Cover letter documents created
Ready to send to: consulting


In [30]:
# Test the updated email function
print("=== UPDATED EMAIL CONTENT ===")
print(email_content)
print("\n" + "="*50)

=== UPDATED EMAIL CONTENT ===
Dear OKX Team,

I am enthusiastic about the Graduate Software Engineer opportunity at OKX, especially your mission to democratize financial services through cutting-edge technology.

With First Class Honors in my studies, I bring hands-on experience from multiple software engineer internships where I developed full-stack applications, optimized system performance, and worked with large datasets. My technical skills in software development and data analysis, combined with strong project leadership experience, make me well-suited for this role.

I am confident that my technical qualifications, practical experience, and passion for fintech make me an excellent candidate for this position. I would welcome the opportunity to discuss how I can contribute to OKX's continued success.

Thank you for your consideration. Attached you will find my CV and supporting documents.

Best regards,
Felix Lau Pangestu


